In [8]:
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from networkx.drawing.nx_agraph import write_dot, graphviz_layout

#test case
from testerMF.testerMF import Tester 
testcase = Tester(userId='F14DM', day='4')

Соединение установлено


In [1]:
!python3 -m pip install networkx --user

     |████████████████████████████████| 1.6MB 834kB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


### Шаг 1 
### Создадим дерево

`Решающее дерево (Decision tree)` — решение задачи обучения с учителем, основанный на том, 
как решает задачи прогнозирования человек. 

В общем случае — это k-ичное дерево с решающими правилами в нелистовых вершинах (узлах) и некотором заключении о целевой функции в листовых вершинах (прогнозом). 

Решающее правило — некоторая функция от объекта, позволяющее определить, в какую из дочерних вершин нужно поместить рассматриваемый объект. 

В листовых вершинах могут находиться разные объекты: класс, который нужно присвоить попавшему туда объекту (в задаче классификации), вероятности классов (в задаче классификации), непосредственно значение целевой функции (задача регрессии).

![](img/Decision_tree.png)

=========

Построить дерево решенй, можно самому, а можно использую библиотеку для создания и манимулирования структурами.

Я предлагаю Вам потратить меньше времени на построение дерева и использовать библиотеку [https://networkx.github.io/documentation/stable/index.html](NetwokX)

[https://networkx.github.io/documentation/stable/reference/classes/digraph.html](nx.DiGraph, который будет использоватся у меня в классе)

*Вы можете попробовать использоват дерево из Random Forest*

### Решающие правила

Чаще всего в качестве $ Q_t(x) $ берут просто один из признаков, то есть $ x^{i(t)} $.

Традиционные разбиения на диапазоны:

$ S_t(j) = \left \{x \in \mathbb{X}: h_j \le x^{i(t)} \le h_{j+1}\right \} $ для выбранных $ h_1, \dots, h_{j+1} $

$ S_t(1) = \left \{x \in \mathbb{X}: \langle x,v \rangle \le 0\right \}; S_t(2) = \left \{x \in \mathbb{X}: \langle x,v \rangle > 0\right \} $ — по сути проверка угла.

$ S_t(1) = \left \{x \in \mathbb{X}: \rho (x, x_0) \le h\right \}; S_t(2) = \left\{x \in \mathbb{X}: \rho (x, x_0) > h\right \} $, где расстояние $ \rho $ определено в некотором метрическом пространстве (например,$ \rho (x, y) = |x - y| $).

$ S_t(1) = \left \{x \in \mathbb{X}: x^{i(t)} \le h\right \}; S_t(2) = \left \{x \in \mathbb{X}: x^{i(t)} > h\right \} $ — предикаты, $ \langle x, v \rangle $ — скалярное произведение векторов.

### Критерии остановки построения дерева

Ограничение максимальной глубины дерева.

Ограничение минимального числа объектов в листе.

Ограничение максимального количества листьев в дереве.

Останов в случае, если все объекты в вершине относятся к одному классу.

Требование, что Information gain при дроблении улучшался как минимум на $ s $ процентов.

### Выбор прогноза в листе

Самый простой способ — взять самый часто встречающийся класс среди объектов обучающей выборки, попавших в этот лист, для классификации или среднее целевых функций этих объектов для регрессии.

В задачах классификации c $ k $ классами в листе $ t $ также можно хранить вероятности классов — например, по классической вероятности: $ \mathbb{P}(y=y_i|x) = \dfrac{\#{X_t}}{N(t)} $, где $ \mathbb{Y} = \{y_1, y_2, \dots, y_k\} $ — классы. 

Если задана матрица штрафов, то есть в случае несимметричных потерь, можно минимизировать штраф и взять в качестве класса в листе $ \hat{y} $:

$ \hat{y} = \underset{y \in \mathbb{Y}}{\operatorname{argmin}} \sum_{i} \lambda_{y_{i}y} $, где $ \lambda_{y_iy} $ — элементы матрицы штрафов.

В регрессии — использовать функцию потерь, то есть:

$ \hat y = \underset{y \in \mathbb{Y}}{\operatorname{argmin}}\sum_{i} L(y, y_i) $


In [11]:
class decisiontree(nx.DiGraph):
    def __init__(self):
        super(decisiontree, self).__init__()
        """
        добавим node в качестве вершины дерева
        
        """
        self.add_node('root', nodetype='fork', feature=None, splitval=None, weight=None)
        
        
    def listnodes(self):
        return self.nodes
        
    def _splitnode(self, node):
        if node is 'root':
            lname, rname = 'L', 'R'
        else:
            lname, rname = node + 'L', node + 'R'
            
        """
        добавим nodes и edges для левого и правого разбиения
        """
        self.add_node(lname, nodetype=None, feature=None, splitval=None, weight=None)
        self.add_node(rname, nodetype=None, feature=None, splitval=None, weight=None)
        self.add_edge(node, lname)
        self.add_edge(node, rname)
        
        
    def _lossredux(self, dlsum, ddlsum, dlsum_L, ddlsum_L, dlsum_R, ddlsum_R, lamb):
        """
        Оценим потерю
        
        сумма по листьям (sqr(leaf) / (leaf + gamma)) - резуьтат вершины
        
        """
        term1 = np.square(dlsum_L) / (ddlsum_L + lamb)
        term2 = np.square(dlsum_R) / (ddlsum_R + lamb)
        term3 = np.square(dlsum) / (ddlsum + lamb)
        return term1 + term2 - term3
    
    def _leafweight(self, dl, ddl, lamb):
        """
        сумма результатов в листе / (сумма результатов в листе + lambda)
        """
        return np.sum(dl) / (np.sum(ddl) + lamb) 
        
        
    def build(self, node, X, dl, ddl, shrinkrate, depth, params):
        assert len(list(self.successors(node) ) ) == 0
        
        if depth > params['maxdepth']:
            self.nodes[node]['nodetype'] = 'leaf'
            self.nodes[node]['weight'] = self._leafweight(
                dl, ddl, params['lambda'])*shrinkrate
            return
            
        dlsum, ddlsum = np.sum(dl), np.sum(ddl)
        
        best_lossredux = 0.
        best_splitval = 0.
        best_feature = None
        best_idx_L, best_idx_R = None, None
        
        for feature in X.columns:
            dlsum_L, ddlsum_L = 0., 0.
            sorted_idx = X[feature].argsort()

            for i, idx in enumerate(sorted_idx):
                dlsum_L += dl[idx]
                ddlsum_L += ddl[idx]
                dlsum_R, ddlsum_R = dlsum - dlsum_L, ddlsum - ddlsum_L
                
                lossredux = self._lossredux(dlsum, ddlsum, dlsum_L, ddlsum_L, \
                                            dlsum_R, ddlsum_R, params['lambda'])

                if lossredux > best_lossredux:
                    best_lossredux = lossredux
                    best_feature = feature
                    
                    """
                    Разобьем фичи и получим лучшие
                    """
                    
                    best_splitval = X[feature].iloc[idx]
                    best_idx_L = sorted_idx[:i+1]
                    best_idx_R = sorted_idx[i+1:]

        if best_lossredux < params['min_lossredux']:
            self.nodes[node]['nodetype'] = 'leaf'
            self.nodes[node]['weight'] = self._leafweight(
                dl, ddl, params['lambda'])*shrinkrate
        else:
            self.nodes[node]['nodetype'] = 'fork'
            self.nodes[node]['splitval'] = best_splitval
            self.nodes[node]['feature'] = best_feature
            
            self._splitnode(node)
    
            """
            Рекурсивно углубимся по дереву
            
            """
            self.build(list(self.successors(node))[0],
                            X.iloc[best_idx_L],
                            dl[best_idx_L],
                            ddl[best_idx_L],
                            shrinkrate,
                            depth+1,
                            params)
            self.build(list(self.successors(node))[1],
                            X.iloc[best_idx_R],
                            dl[best_idx_R],
                            ddl[best_idx_R],
                            shrinkrate,
                            depth+1,
                            params)
        
            
    def run(self, x, node):
        if self.nodes[node]['nodetype'] is 'leaf':
            return self.nodes[node]['weight']
        else:
            if x[self.nodes[node]['feature']] <= self.nodes[node]['splitval']:
                return self.run(x, list(self.successors(node) )[0])
            else:
                return self.run(x, list(self.successors(node) )[1])

#### Тестирование дерева

In [14]:
#Загрузка данных
features = ['sex', 'length', 'diameter', 'height', 'weight', \
            'shuck_weight', 'visc_weight', 'shell_weight']

labels = ['rings']

intsex = {'M':0, 'F':1, 'I':2}

#только 10 строк
data = pd.read_csv('data/abalone.data', names = features + labels, nrows = 10)

data['sex'] = data['sex'].replace(intsex)
data.loc[(data['rings'] < 10), 'rings'] = 0
data.loc[(data['rings'] >= 10), 'rings'] = 1

for x in features:
    data.loc[:, x] *= np.divide(1, data[x].max())

#Подготовка X|Y
Xtrain, Ytrain = data[features], data[labels]

#Игрушечные данные для проверке тест
l = np.array(Ytrain.values)
dl = np.array((2 * Ytrain).values)
ddl = np.full(len(Ytrain), 2)
params = {'gamma': 0.,
          'lambda': 1.,
          'maxdepth': 5,
          'learnrate': 0.3,
          'shrinkrate': 1.0,
          'earlystop_rounds': 4,
          'earlystop_precision': 1e-5,
          'min_lossredux': 0.1}

#инициализация функции
newtree = decisiontree()
print("Tree with node", newtree.degree)
newtree.build('root', Xtrain, dl, ddl, params['shrinkrate'], 0, params)
print("Predict by Tree = ", newtree.run(Xtrain.all(), 'root'))
#функция проверки

#функция провеки
# testcase.sendTask(part='2', task='1', func=newtree.run(Xtrain.all(), 'root'))

Tree with node [('root', 0)]
Predict by Tree =  0.7692307692307693
Решение отправлено на проверку
Начало тест кейса, получаем данные...
Test Case не пройден
Входные данные - {'X':                                                    0  \
0  [0.0, 0.8272727272727273, 0.8295454545454546, ...   
1                                                sex   

                                                   1  \
0  [0.0, 0.6363636363636364, 0.6022727272727274, ...   
1                                             length   

                                                   2  \
0  [0.5, 0.9636363636363636, 0.9545454545454546, ...   
1                                           diameter   

                                                   3  \
0  [0.0, 0.7999999999999999, 0.8295454545454546, ...   
1                                             height   

                                                   4  \
0  [1.0, 0.6, 0.5795454545454546, 0.5333333333333...   
1                               

### А ТЕПЕРЬ ~ПОЖЕСТИМ~ *ПОБУСТИМ*

![](img/algs.png)

### Добавим метрики оценки результатов

Нашей основной функций будет MSE

![](img/mse_1.jpg)

In [46]:
# добавим метрики оценки
maxloss = 1.0e15

## Quantum Relative Entropy
def qrelative_entropy(y, output):
    assert y.shape == output.shape
    eps = 1e-16
    y = np.piecewise(y,[y<eps, y>=eps], [lambda x: eps, lambda x: x])
    
    if output is None:
        output = np.random.uniform(eps, 1-eps, size = y.shape)
    else:
        output = np.piecewise(output,[output<eps, output>=eps], [lambda x: eps, lambda x: x])
    
    S = (1/len(y))*np.sum([y[i]*np.log(y[i]/output[i]) for i in range(len(y))])
    dS = -(1/len(y))*np.array([y[i]/output[i] for i in range(len(y))]).reshape(len(y), 1)
    ddS = (1/len(y))*np.array([y[i]/output[i]**2 for i in range(len(y))]).reshape(len(y), 1)
    
    return S, dS, ddS

## Mean-squared Error
def mse(y, output):
    
    """
    y - факт
    output - рузультат функции score
    
    """
    
    assert y.shape == output.shape
    """
    напишим MSE
    """
    l = np.sum(np.power(output - y, 2)) / len(y)
    dl = 2 / len(y) * output.reshape((-1, 1))
    ddl = 2 / len(y) * np.ones((len(y), 1))
    return l, dl, ddl


In [57]:
#создадим градиентный спуск над решающими деревьями
class xgbdt:
    def __init__(self, params):
        self.params = params
        self.trees = list()
        self.best_run = None

    def _newtree(self, Xtrain, dl, ddl, shrinkrate):
        
        """
        Создадим одно дерево
        """
        newtree = decisiontree()
        newtree.build('root', Xtrain, dl, ddl, shrinkrate, 0, params)
        return newtree
    
    def _scores(self, X):
        """
        Получим скор из дерева
        """
        scores = np.zeros(len(X))
        for i in range(len(X)):
            scores = self._preddict(X.iloc[i])
        return scores
    
    def _preddict(self, x):
        assert len(self.trees) > 0
        return np.sum([tree.run(x, 'root') for tree in self.trees])

    def _loss(self, y, output, losstype = 'mse'):
        y = y.values.reshape(output.shape)
        if losstype is 'mse':
            l, dl, dll = mse(y, output)
        return l, dl, dll

    def cleartrees(self):
        self.trees.clear()
    
    def train(self, train_set, valid_set, features, labels, nboost):
        print('Starting training')
        start_time = time.time()

        shrinkrate = self.params['shrinkrate']
        v_bestloss = maxloss
        best_run = None
        
        Xtrain, Ytrain = train_set[features], train_set[labels]
        Xvalid, Yvalid = valid_set[features], valid_set[labels]
        
        for iboost in range(nboost):
            boost_time = time.time()
            
            if iboost == 0:
                # rand = np.random.RandomState(1234)
                dloss = np.random.uniform(size = train_set.shape[0])
                ddloss = np.full(dloss.shape[0], 2)
                
            newtree = self._newtree(Xtrain, dloss, ddloss, shrinkrate)
            self.trees.append(newtree)
            
            if iboost > 0:
                shrinkrate *= self.params['learnrate']
            
            #сделаем шаг бустинга
            
            scores = self._scores(Xtrain)
            loss, dloss, ddloss = self._loss(Ytrain, scores)
            
            v_scores = self._scores(Xvalid)
            v_loss, _, _ = self._loss(Yvalid, vscores)
            
            loss = np.mean(loss)
            v_loss = np.mean(v_loss)
            
            print("Iter {:>3}, Train Loss: {:.10f}, Valid Loss: {:.5f}, Time: {:.2f} secs"
                  .format(iboost, loss, v_loss, time.time() - boost_time))
            
            if v_loss < v_bestloss and v_bestloss - v_loss >= self.params['earlystop_precision']:
                v_bestloss = v_loss
                best_run = iboost
                
            if best_run is not None and iboost - best_run >= self.params['earlystop_rounds']:
                print("Stopped early with best run:")
                print("Iter {:>3}, Valid Loss: {:.10f}".format(best_run, v_bestloss))
                break

        self.best_run = best_run
        print("Training finished after {:.2f} secs".format(time.time() - start_time))

###  Тест на всем наборе данных

In [58]:
features = ['sex', 'length', 'diameter', 'height', 'weight', \
            'shuck_weight', 'visc_weight', 'shell_weight']
labels = ['rings']
intsex = {'M':0, 'F':1, 'I':2}

data = pd.read_csv('data/abalone.data', names = features + labels)

data['sex'] = data['sex'].replace(intsex)
data.loc[(data['rings'] < 10), 'rings'] = 0
data.loc[(data['rings'] >= 10), 'rings'] = 1

for x in features:
    data.loc[:, x] *= np.divide(1, data[x].max())
    
df_train, df_valid = data.iloc[:3000], data.drop(data.iloc[:,3000:])


df_train = df_train
df_valid = df_valid

In [59]:
params = {'gamma': 0.,
          'lambda': 1.,
          'maxdepth': 5,
          'learnrate': 0.3,
          'shrinkrate': 1.0,
          'earlystop_rounds': 4,
          'earlystop_precision': 1e-5,
          'min_lossredux': 0.1}

gbt = xgbdt(params)
gbt.train(df_train, df_valid, features, labels, nboost = 100)

Starting training


ValueError: cannot reshape array of size 3000 into shape ()

In [ ]:
from sklearn.metrics import roc_auc_score

print('Score is ', round(roc_auc_score(df_valid['rings'], gbt._scores(df_valid.drop('rings', axis = 1))),2))

#проверка
testcase.sendTask(part='2', task='2', func=gbt._scores(df_valid.drop('rings', axis = 1)))